# SINet COD10K Detection
Implementazione migliorata di SINet per il rilevamento di oggetti mimetizzati.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
from PIL import Image
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F

In [2]:
device = torch.device('cuda')

In [3]:
class CODDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_folder)
                            if os.path.isfile(os.path.join(mask_folder, f.replace('.jpg', '.png')))]
        if len(self.image_files) == 0:
            raise ValueError(f"Nessuna immagine valida trovata in {image_folder} con maschere corrispondenti in {mask_folder}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        mask_path = os.path.join(self.mask_folder, self.image_files[idx].replace('.jpg', '.png'))
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        return image, mask

In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean() if self.reduction == 'mean' else F_loss.sum()

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.GaussianBlur(kernel_size=(3, 3)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train_dataset = CODDataset(r'C:\Users\Nicholas\Desktop\COD10K-v3\Train\Image',
                           r'C:\Users\Nicholas\Desktop\COD10K-v3\Train\GT_Object',
                           transform=transform)
test_dataset = CODDataset(r'C:\Users\Nicholas\Desktop\COD10K-v3\Test\Image',
                          r'C:\Users\Nicholas\Desktop\COD10K-v3\Test\GT_Object',
                          transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

In [7]:
class SINet(nn.Module):
    def __init__(self):
        super(SINet, self).__init__()
        self.backbone = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.DEFAULT)
        self.backbone_features = self.backbone.features
        self.edge_attention = nn.Sequential(
            nn.Conv2d(1792, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1792 + 256, 512, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        features = self.backbone_features(x)
        edge_map = self.edge_attention(features)
        edge_map = torch.nn.functional.interpolate(edge_map, size=features.shape[2:], mode='bilinear', align_corners=False)
        combined = torch.cat((features, edge_map.expand(-1, 256, -1, -1)), dim=1)
        output = self.decoder(combined)
        return output

In [8]:
model = SINet().to(device)
criterion = FocalLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

: 

In [ ]:
for epoch in range(5):
    model.train()
    total_loss = 0
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        preds = model(images)
        preds = nn.functional.interpolate(preds, size=masks.shape[2:], mode='bilinear', align_corners=False)
        loss = criterion(preds, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}')

In [ ]:
torch.save(model.state_dict(), 'sinet_cod_model.pth')

In [ ]:
def compute_iou(mask_np, pred_np):
    if mask_np.sum() == 0:
        return None
    return jaccard_score(mask_np, pred_np, zero_division=1)

def visualize_predictions(model, dataloader, num_images=10):
    model.eval()
    iou_scores = []
    with torch.no_grad():
        for i, (image, mask) in enumerate(dataloader):
            if i >= num_images:
                break
            image = image.unsqueeze(0).to(device)
            mask = mask.to(device)
            pred = model(image)
            pred = (pred > 0.5).float()
            mask_np = (mask.squeeze().cpu().numpy() > 0.5).astype(int)
            pred_np = (pred.squeeze().cpu().numpy() > 0.5).astype(int)
            iou = compute_iou(mask_np, pred_np)
            if iou is not None:
                iou_scores.append(iou)
    print(f"Mean IoU (filtered): {sum(iou_scores) / len(iou_scores):.2f}")

visualize_predictions(model, test_loader, num_images=10)